In [ ]:
%pip install langchain-openai

In [1]:
%env OPENAI_API_KEY=sk-7nEgUqufIHJhyUgfO2DzfV22uoS9AKTQ1lPQ96WoLDdTKg1b
%env OPENAI_URL=https://api.chatanywhere.tech/v1
import os
key=os.getenv('OPENAI_API_KEY')
url=os.getenv('OPENAI_URL')

env: OPENAI_API_KEY=sk-7nEgUqufIHJhyUgfO2DzfV22uoS9AKTQ1lPQ96WoLDdTKg1b
env: OPENAI_URL=https://api.chatanywhere.tech/v1


In [18]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
model_name='gpt-3.5-turbo'
model=ChatOpenAI(base_url=url,api_key=key,model=model_name)

prompt=ChatPromptTemplate.from_template('讲一个笑话,跟{topic}相关')

output=StrOutputParser()

chain=prompt|model|output

chain.invoke({'topic','猪'})



'为什么猪不会玩扑克牌？\n\n因为他们总是把手中的牌全都揭露出来！'

### Runables
Interface
- stream:stream back chunks of the response [流媒体返回]
- invoke:call the chain on an input
- batch: call the chain on a list of inputs

Component
- Prompt > Dict> PromptValue
- ChatModel > str,list of chat messages,PromptValue > ChatMessage
- LLM > str,list of chat messages,PromptValue > str
- OutputParser >  The output of an LLM or ChatModel > Depends on the parser
- Retriever > Single string  > List of Documents
- Tool > Single string or dictionary, depending on the tool > 
Depends on the tool 

Schemas
- input_schema
- output_schema

> https://python.langchain.com/v0.1/docs/expression_language/primitives/sequence/




In [19]:
prompt.input_schema.schema()

{'title': 'PromptInput',
 'type': 'object',
 'properties': {'topic': {'title': 'Topic', 'type': 'string'}}}

In [20]:
model.input_schema.schema()

{'title': 'ChatOpenAIInput',
 'anyOf': [{'type': 'string'},
  {'$ref': '#/definitions/StringPromptValue'},
  {'$ref': '#/definitions/ChatPromptValueConcrete'},
  {'type': 'array',
   'items': {'anyOf': [{'$ref': '#/definitions/AIMessage'},
     {'$ref': '#/definitions/HumanMessage'},
     {'$ref': '#/definitions/ChatMessage'},
     {'$ref': '#/definitions/SystemMessage'},
     {'$ref': '#/definitions/FunctionMessage'},
     {'$ref': '#/definitions/ToolMessage'}]}}],
 'definitions': {'StringPromptValue': {'title': 'StringPromptValue',
   'description': 'String prompt value.',
   'type': 'object',
   'properties': {'text': {'title': 'Text', 'type': 'string'},
    'type': {'title': 'Type',
     'default': 'StringPromptValue',
     'enum': ['StringPromptValue'],
     'type': 'string'}},
   'required': ['text']},
  'ToolCall': {'title': 'ToolCall',
   'type': 'object',
   'properties': {'name': {'title': 'Name', 'type': 'string'},
    'args': {'title': 'Args', 'type': 'object'},
    'id': {

In [23]:
# stream 返回响应块
for s in chain.stream({'topic':'bear'}):
    print(s,end='',flush=True)

chain.invoke({'topic','bear'})

# 批量处理
chain.batch([{'topic':'bears'},{'topic':'cats'}])

为什么熊在冬眠时不会感到饥饿呢？

因为他们吃得太多，肚子里都装满了！

In [28]:
# parallesim 排比,RunnableParallel并行执行
# RunnablePassthrough传递invoke的参数
from langchain_core.runnables import RunnableParallel,RunnablePassthrough

first=RunnableParallel({'context':RunnablePassthrough(),'question':RunnablePassthrough()})

second=ChatPromptTemplate.from_template('{context},{question}')

chain1=first|second

chain1.invoke('21')


ChatPromptValue(messages=[HumanMessage(content='21,21')])

RunnableParallel primitive is essentially a dict whose values are runnables (or things that can be coerced to runnables, like functions). It runs all of its values in parallel, and each value is called with the overall input of the RunnableParallel. The final return value is a dict with the results of each value under its appropriate key
> primitive 原始

RunnableParallel are also useful for running independent processes in parallel, since each Runnable in the map is executed in parallel.

In [ ]:
# 参数绑定
from operator import itemgetter

from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

vectorstore = FAISS.from_texts(
    ["harrison worked at kensho"], embedding=OpenAIEmbeddings()
)
retriever = vectorstore.as_retriever()

template = """Answer the question based only on the following context:
{context}

Question: {question}

Answer in the following language: {language}
"""
prompt = ChatPromptTemplate.from_template(template)

chain = (
    {
        "context": itemgetter("question") | retriever,
        "question": itemgetter("question"),
        "language": itemgetter("language"),
    }
    | prompt
    | model
    | StrOutputParser()
)

chain.invoke({"question": "where did harrison work", "language": "italian"})

bind函数
RunnablePassthrough 数据传输
RunnaleLambda 运行自定义函数
RunnablePassthrough.assgin:takes an input value and adds the extra arguments passed to the assign function

In [ ]:
# RunnablePassthrough.assgin()

from langchain_core.runnables import RunnableParallel, RunnablePassthrough

runnable = RunnableParallel(
    extra=RunnablePassthrough.assign(mult=lambda x: x["num"] * 3),
    modified=lambda x: x["num"] + 1,
)

runnable.invoke({"num": 1})

### Logging

```python
import os

os.environ["LANGCHAIN_API_KEY"] = "..."
os.environ["LANGCHAIN_TRACING_V2"] = "true"

anthropic_chain.invoke("ice cream")
```

### Inspect your runnable
get_graph().print_ascii():get a graph of the runnable and print

get_prompts() get prompts in chain


In [27]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.messages import HumanMessage, SystemMessage
model='gpt-4-turbo'
ai=ChatOpenAI(base_url=url,api_key=key,model=model)

prompt=ChatPromptTemplate.from_messages([HumanMessage('ai agent是什么?langchain agent模块又有什么用处')])

ai.invoke(prompt.format_messages())

AIMessage(content='"AI agent"（人工智能代理）是指在一定环境中，能够自主地进行感知、学习、决策和执行行动的计算机系统。这种代理能够根据输入信息（感知环境或从用户接收的信息）进行处理，并做出相应的反应或决策，以达到特定的目标或任务。AI代理广泛应用于各种领域，如自动驾驶车辆、智能家居系统、游戏AI、自动交易系统等。\n\n关于"LangChain agent"，这个名词似乎是指在一个特定的技术、库或框架中的一个模块或组件，但基于现有信息库并没有详细的公开信息。如果"LangChain"是指某个具体的技术或产品，在没有更多上下文的情况下，难以准确解释"LangChain agent模块"的具体用途。通常，类似这样的模块可能涉及语言处理、链式决策支持、自然语言理解等功能，用于提升AI系统处理语言相关任务的能力。\n\n建议查阅更具体的技术文档或官方资源以获得详细和准确的信息。如果你有更多关于"LangChain"的背景信息，我可以尝试提供更具体的解答。', response_metadata={'token_usage': {'completion_tokens': 388, 'prompt_tokens': 28, 'total_tokens': 416}, 'model_name': 'gpt-4-turbo', 'system_fingerprint': 'fp_294de9593d', 'finish_reason': 'stop', 'logprobs': None}, id='run-8819ddc9-3bbe-45ec-8f58-b67b625a1caa-0')